# AI Doc Checker - Evaluation System V2

Complete evaluation system for student PDF submissions with SAP BW content.

## System Overview

1. **PDF Processing**: Extract images from student PDF submissions
2. **Classification**: Use EfficientNet to classify images into 6 categories
3. **Evaluability Check**: Use Qwen to check if images are suitable for evaluation
4. **Metadata Matching**: Find best reference solutions based on metadata similarity
5. **Detailed Evaluation**: Compare student work with reference and provide scoring (0-100)
6. **Final Result**: Overall score with pass/fail (≥70 = bestanden)


In [1]:
import os
import sys
import json
from datetime import datetime

# Add parent directory to path for imports
sys.path.append('..')

from evaluation_engine import EvaluationEngine
from metadata_generator import MetadataGenerator
from qwen_client import QwenClient
from image_classifier import ImageClassifier
from pdf_processor import PDFImageExtractor


In [2]:
# Test Qwen connection
qwen_client = QwenClient()
health = qwen_client.health_check()

print("Qwen Server Status:")
print(f"Status: {health.get('status')}")
print(f"Model Loaded: {health.get('model_loaded')}")
print(f"CUDA Available: {health.get('cuda_available')}")

if not health.get('model_loaded'):
    print("Qwen server is not ready. Please check the server.")
else:
    print("Qwen server is ready for evaluation")


Qwen Server Status:
Status: healthy
Model Loaded: True
CUDA Available: True
Qwen server is ready for evaluation


In [3]:
# Initialize classifier
try:
    classifier = ImageClassifier(model_path="../model/efficientnet_b0_best.pth")
    print("EfficientNet classifier loaded")
    print(f"Categories: {classifier.class_names}")
    print(f"Confidence threshold: {classifier.confidence_threshold}")
except Exception as e:
    print(f"Failed to load classifier: {e}")


✅ EfficientNet model loaded from ../model/efficientnet_b0_best.pth
EfficientNet classifier loaded
Categories: ['Data Source', 'Data-Flow', 'Data-Transfer-Process', 'Excel-Tabelle', 'Info-Object', 'Transformation']
Confidence threshold: 0.6


In [4]:
# Initialize evaluation engine
metadata_db_path = "metadata_database.json"

try:
    engine = EvaluationEngine(metadata_db_path=metadata_db_path)
    print("Evaluation engine initialized and ready")
except Exception as e:
    print(f"Failed to initialize evaluation engine: {e}")
    print("Make sure the metadata database exists and Qwen server is running.")


✅ EfficientNet model loaded from ./model/efficientnet_b0_best.pth
✅ Port 5000 is accessible (SSH tunnel likely active)
✅ Qwen client connected and ready
   Status: healthy
   Model Loaded: True
   CUDA Available: True
🔄 Loading metadata database from: metadata_database.json
✅ Metadata database loaded:
   Total categories: 6
   - Data Source: 68 images
   - Data-Flow: 44 images
   - Data-Transfer-Process: 42 images
   - Excel-Tabelle: 132 images
   - Info-Object: 190 images
   - Transformation: 39 images
✅ Evaluation Engine initialized
Evaluation engine initialized and ready


In [5]:
# Evaluate a student PDF submission
test_pdf_path = r"C:\Users\egese\Documents\GitHub\ai-doc-checker\test_abgabe.pdf"  # Replace with actual PDF path

if 'engine' in locals() and os.path.exists(test_pdf_path):
    print("Starting full evaluation...")
    print("=" * 50)
    
    # Run evaluation
    result = engine.evaluate_pdf_submission(test_pdf_path)
    
    print("\n" + "=" * 50)
    print("EVALUATION COMPLETED")
    print("=" * 50)
    
    # Display summary
    summary = engine.get_evaluation_summary(result)
    print(summary)
    
    # Save result
    output_file = engine.save_evaluation_result(result)
    print(f"\nFull results saved to: {output_file}")
    
else:
    print(" Cannot run full evaluation:")
    if 'engine' not in locals():
        print("- Evaluation engine not initialized")
    if not os.path.exists(test_pdf_path):
        print(f"- Test PDF not found: {test_pdf_path}")
        print("- Place a test PDF at the specified path to run evaluation")


Starting full evaluation...
🔄 Extracting images from PDF...
✅ Extracted 1 images
🔄 Classifying images...
  ✅ page_1_img_1.png: Excel-Tabelle (0.997)
🔄 Checking image evaluability...
  ✅ page_1_img_1.png: Evaluable
🔄 Performing detailed evaluations...
🔍 Finding top 1 references for category: 'Excel-Tabelle'
   Available categories: ['Data Source', 'Data-Flow', 'Data-Transfer-Process', 'Excel-Tabelle', 'Info-Object', 'Transformation']
   Found 132 reference images for 'Excel-Tabelle'
  🔍 Evaluating against 1 references...
    ⚠️ Reference 67f6b67affc04f8899b5ab8867eaabb7.png not found, skipping...
  ❌ No successful evaluations for page_1_img_1.png

EVALUATION COMPLETED

📊 EVALUATION SUMMARY

PDF: test_abgabe.pdf
Date: 2025-07-04T21:41:15.255171

📈 RESULTS:
- Images extracted: 1
- Valid classifications: 1
- Successful evaluations: 0
- Overall Score: 0.0/100
- Result: ❌ NICHT BESTANDEN

📝 DETAILED SCORES:

⚠️  ERRORS:
- No successful evaluations

💾 Evaluation result saved to: evaluation_re

In [ ]:
print("SYSTEM STATUS SUMMARY")

# Check all components
components = {
    "Qwen Server": 'qwen_client' in locals() and qwen_client.health_check().get('model_loaded'),
    "EfficientNet Classifier": 'classifier' in locals(),
    "Metadata Database": os.path.exists(metadata_db_path),
    "Evaluation Engine": 'engine' in locals(),
    "PDF Processor": True  # Always available
}

for component, status in components.items():
    status_icon = "✅" if status else "❌"
    print(f"{status_icon} {component}")

all_ready = all(components.values())
print(f"\nSystem Ready: {'YES' if all_ready else 'NO'}")

if all_ready:
    print("\nThe evaluation system is fully operational!")
    print("You can now process student PDF submissions.")
else:
    print("\nSome components need attention:")
    for component, status in components.items():
        if not status:
            print(f" - {component}")


SYSTEM STATUS SUMMARY
✅ Qwen Server
✅ EfficientNet Classifier
✅ Metadata Database
✅ Evaluation Engine
✅ PDF Processor

System Ready: YES

The evaluation system is fully operational!
You can now process student PDF submissions.


: 